In [1]:
import os
import time

import lightgbm as lightgbm
import pandas as pd
import numpy as np
from xgboost import XGBRegressor

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, VotingRegressor, AdaBoostRegressor, \
    BaggingClassifier, ExtraTreesClassifier, GradientBoostingRegressor

from sklearn.mo del_selection import StratifiedKFold, train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pylab as plt
import pickle

In [ ]:

changepoints = ['2019-06-07 00:00', '2019-07-04 00:00', '2019-08-13 00:00']
from sklearn.metrics import f1_score


chosen_stations = ['253200080', '353200272']
X_train, X_test, y_train_labels = pickle.load(open( "./train_2C_gprs_x.pkl", "rb" )),pickle.load(open( "./test_2C_gprs_x.pkl", "rb" )), pickle.load(open( "./train_y.pkl", "rb" ))
X_test.index=X_test.index.strftime('%Y-%m-%d %H:%M')


df = X_train[['253200080_Precipitation-sum-past-1-hrs', '353200272_Precipitation-sum-past-1-hrs','value']].copy()
df=df[::-1]
df['253200080'] = df['253200080_Precipitation-sum-past-1-hrs'].shift(1).rolling(24).sum()
df['353200272'] = df['353200272_Precipitation-sum-past-1-hrs'].shift(1).rolling(24).sum()
df = df[::-1]
df['rain'] = df[["253200080", "353200272"]].max(axis=1)
chosen_stations = ['253200080', '353200272']

X_train, y_train = X_train.iloc[:-24], df['rain'].iloc[:-24]

for cp in changepoints:
    i = X_train.loc[:cp].shape[0]
    X_train = X_train.drop(X_train.index[i-24:i])
    y_train = y_train.drop(y_train.index[i-24:i])

model=XGBRegressor()

X_train_, X_val, y_train_, y_val = train_test_split(X_train, y_train, test_size=0.33, shuffle=False)

scaler = MinMaxScaler()
X_train_ = scaler.fit_transform(X_train_)
X_val = scaler.transform(X_val)
model.fit(X_train_, y_train_)

y_val_pred = model.predict(X_val)
y_val_labels = (y_val > 15).astype(int) 

plt.plot(y_val.values)
plt.plot(y_val_pred)
plt.show()


best_th = 15
best_f1 = 0.
for th in [1,5,7,10,12,15]:
    y_val_labels_pred = (y_val_pred > th).astype(int)
    f1 = f1_score(y_val_labels_pred, y_val_labels)
    print(th, f1) 
    if f1 > best_f1:
        best_th = th
        best_f1 = f1



scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
        
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
pred_labels = (y_pred > th).astype(int)


print(pred_labels.sum())
pd.DataFrame(index = X_test.index, data = pred_labels).to_csv('thlab.csv', sep=';', header=None)